In [653]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

In [654]:
data = pd.read_csv('EOP.csv',encoding='utf-8').iloc[:, :4] 
data.columns = ['零件号','日期','需求数量','仓库']
data.head()

,零件号,日期,需求数量,仓库
0,KITPAB100,20221130,1,3002
1,KITPAB100,20211208,2,3002
2,KITPAB100,20220906,2,3002
3,KITPAB100,20211111,2,3002
4,KITPAB100,20230613,1,3002


In [655]:
data = data[~data['仓库'].isin(['8013','8021','8025','8031'])]
# data =data[data['零件号']!='6RD959801E']
data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
new_df = data.groupby(['零件号','日期','仓库'])['需求数量'].sum().reset_index()
new_df['日期'] = pd.to_datetime(new_df['日期'])

In [656]:
# new_df.loc[(new_df['零件号'] == '6RD959801E') & (new_df['日期'] == '2023/3/27')& 
#            (new_df['仓库'] == '1000-1'), '需求数量'] = 6

In [657]:
data_des = new_df.copy()
data_des["year"] = pd.to_datetime(data_des['日期']).dt.year.astype(int)
data_des["month"] = pd.to_datetime(data_des['日期']).dt.month.astype(int)
real_sum_counts = data_des.groupby(['零件号','仓库','year','month']).sum().reset_index()
real_sum_counts.sort_values(by=['零件号','仓库','year','month']).head(2)

,零件号,仓库,year,month,需求数量
0,02E300066L 00V,1000-1,2017,12,4
1,02E300066L 00V,1000-1,2018,1,8


In [658]:
# real_sum_counts[(real_sum_counts['零件号']=='3CC945208A')&(real_sum_counts['仓库']=='1000-1')].tail(10)

In [659]:
import calendar  
import datetime  
  

    
def feature_processing(data,end_time):
    data['合并列'] = data['零件号'] + '_' + data['仓库']
    # 去除多余的列名
    data = data.drop(columns=['零件号', '仓库'])
    data = data.set_index(['index', '合并列'])['需求数量'].unstack()
    data.columns.name = None
    data = data.reset_index()
    new_df = data.copy().rename(columns={'index': 'date'})
    new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
    new_df = new_df.set_index('date')
    new_df = new_df.asfreq('D')
    new_df = new_df.sort_index()
    data_train = new_df[new_df.index <= end_time].copy()
    return data_train


def demods_groby_month(data):
    """天级别聚合month"""
    data_prs = data.reset_index()
    data_prs = data_prs.rename(columns={'index': 'date'})
    data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
    data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
    data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
    data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
    data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
    data_prs_info = data_prs_info.reset_index()
    data_prs_info[['零件号', '仓库代码']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
    data_prs_info.columns =['year','month','零件号','pred_values','仓库']
    return data_prs_info


def find_outliers_3sigma(data):  
    
    mean = sum(data) / len(data)  
    std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  
    if mean<=150:
        thr_sig_num_high  = mean + 3 * std_dev
    else:
        thr_sig_num_high  = mean
        
    thr_sig_num_low  = mean - 1 * std_dev
#     q3 = lambda x: x.quantile(0.75)  
#     q1 = lambda x: x.quantile(0.25)  
#     thr_sig_num_high = q3(x) + 1.5 * (q3(x) - q1(x)) 
#     thr_sig_num_low = q1(x) - 1.5 * (q3(x) - q1(x)) 

    return thr_sig_num_high,thr_sig_num_low




# 生成日期范围
date_range = pd.date_range(start='2022-06-01', end='2023-10-31', freq='D')
# 创建空的DataFrame，准备存储填充后的结果
filled_df = pd.DataFrame()

# 针对每个 SKU 进行填充操作
for sku, group in new_df.groupby(['零件号','仓库']):
    sku_group = group.set_index('日期').reindex(date_range, fill_value=0).reset_index()
    sku_group['零件号'] = sku[0]
    sku_group['仓库'] = sku[1]
    filled_df = filled_df.append(sku_group, ignore_index=True)
    
    
# 生成一个日期对象，表示2023年1月1日  
start_date = datetime.datetime(2023, 1, 1)  
  
# 初始化一个空列表来保存每个月的最后一天  
month_end_dates = []  
  
# 使用 calendar.monthrange() 函数获取每个月的天数  
# 然后加1天，因为我们想要的是当月的最后一天  
for month in range(4, 10):  # 这里 13 是因为一年只有12个月  
    _, num_days = calendar.monthrange(2023, month)  # 获取2023年的日历  
    end_date = start_date.replace(month=month, day=num_days)  # 创建日期对象  
    month_end_dates.append(end_date)  # 添加到列表中  
pred_full_pp = pd.DataFrame()

for date in month_end_dates:  
    end_train = date.strftime("%Y-%m-%d")
    data_train = feature_processing(filled_df,date)
    sku_list = data_train.columns
    # 创建空的字典，用于存储每个时间序列的预测模型
    forecasters = {}
    # 循环遍历每批次SKU，分别训练预测模型
    for i in range(0, len(sku_list), 200):
        batch_skus = sku_list[i:i + 200]
        for sku in batch_skus:
            forecaster = ForecasterAutoreg(  
            regressor=Ridge(random_state=123),  
            lags=27,  
                    )
            # 拟合模型
            forecaster.fit(y=data_train[sku])
            forecasters[sku] = forecaster
        # print(f"Finished training batch {i // self.batch_size + 1}/{len(sku_list) // self.batch_size + 1}")
    # 进行未来预测
    predictions = pd.DataFrame()
    for sku, forecaster in forecasters.items():
        forecast = forecaster.predict(steps=30)
        predictions[sku] = forecast
    predictions[predictions < 0.1] = 0


    his_info = demods_groby_month(data_train)
    thr_std = his_info.groupby(['零件号','仓库'])['pred_values'].apply(find_outliers_3sigma).reset_index()
    thr_std['pred_values_high']= [x[0] for x in thr_std['pred_values']]
    thr_std['pred_values_lower']= [x[1] for x in thr_std['pred_values']]
    for i in ['pred_values_high','pred_values_lower']:
        thr_std[i] =[0 if x<0 else x for x in thr_std[i]]
    thr_std = thr_std.drop(['pred_values'],axis =1 )
    
    

    pred_info = demods_groby_month(predictions)
    pred_info['pred_values'] = pred_info['pred_values'].round(2)
    
    
    full_pred_info = pd.merge(pred_info,thr_std,on =['零件号','仓库'],how ='left')
    full_pred_info['pred_values'] = full_pred_info.apply(lambda row: row['pred_values_high'] 
                                if row['pred_values'] > row['pred_values_high'] 
                                 else (row['pred_values_lower'] if row['pred_values'] < row['pred_values_lower'] 
                                       else row['pred_values']), axis=1)  
#     full_pred_info = full_pred_info.drop(['pred_values_lower','pred_values_high'],axis =1 )
    

    pred_full_pp = pred_full_pp.append(full_pred_info)


pred_full_pp.head()



/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_59614/838568169.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_59614/838568169.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_59614/838568169.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_59614/838568169.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

,year,month,零件号,pred_values,仓库,pred_values_high,pred_values_lower
0,2023,5,02E300066L 00V,0.00,1000-1,10.151803,0.0
1,2023,5,03C907660S,14.41,1000-1,242.171218,0.0
2,2023,5,09G300033L,0.00,1000-2,0.953348,0.0
3,2023,5,09G300055J,0.00,1000-1,0.953348,0.0
4,2023,5,09M300036S,0.00,1000-1,2.122454,0.0


In [660]:
compar_pred_real_info = pd.merge(pred_full_pp,real_sum_counts,on =['零件号','仓库','year','month'],how ='left')
compar_pred_real_info.fillna(0,inplace=True)

In [661]:
svg_pred =pd.read_csv('svg预测结果2.csv',encoding='gb18030')
svg_pred.columns = ['零件代码','仓库','日期','预测值']

In [662]:
svg_pred['日期'] = pd.to_datetime(svg_pred['日期'], format='%Y%m').dt.strftime('%Y-%m')
svg_pred["year"] = pd.to_datetime(svg_pred['日期']).dt.year.astype(int)
svg_pred["month"] = pd.to_datetime(svg_pred['日期']).dt.month.astype(int)
svg_pred = svg_pred.groupby(['零件代码','仓库','year','month'])['预测值'].sum().reset_index()
svg_pred = svg_pred.rename(columns ={"零件代码":"零件号"})

In [663]:
# svg_pred[(svg_pred['零件号']=='03C907660S')&(svg_pred['year']==2023)&(svg_pred['month']==3)]

In [664]:
full_compart_info  = pd.merge(compar_pred_real_info,svg_pred,on = ['零件号','仓库','year','month'],how ='left')

full_compart_info.fillna(0,inplace =True)

full_compart_info['需求数量']= [1.01 if x ==0 else x for x in full_compart_info['需求数量']]

full_compart_info['pred_values']= [1 if x ==0 else x for x in full_compart_info['pred_values']]

full_compart_info['chumi_mape'] = (abs(full_compart_info['需求数量'] - full_compart_info['pred_values']) 
                                 / full_compart_info['需求数量']) 
full_compart_info['svg_mape'] = (abs(full_compart_info['需求数量'] - full_compart_info['预测值']) 
                                 / full_compart_info['需求数量']) 
full_compart_info.sort_values(['零件号','month']).head(20)

,year,month,零件号,pred_values,仓库,pred_values_high,pred_values_lower,需求数量,预测值,chumi_mape,svg_mape
0,2023,5,02E300066L 00V,1.00,1000-1,10.151803,0.0,1.01,0.217825,0.009901,0.784332
24,2023,6,02E300066L 00V,1.00,1000-1,9.842194,0.0,1.01,0.130700,0.009901,0.870594
48,2023,7,02E300066L 00V,1.00,1000-1,9.548455,0.0,1.00,0.078368,0.000000,0.921632
72,2023,8,02E300066L 00V,0.22,1000-1,9.236574,0.0,1.01,2.376667,0.782178,1.353136
96,2023,9,02E300066L 00V,1.00,1000-1,8.987869,0.0,1.01,1.980000,0.009901,0.960396
120,2023,10,02E300066L 00V,1.00,1000-1,8.753803,0.0,1.01,1.653333,0.009901,0.636963
1,2023,5,03C907660S,14.41,1000-1,242.171218,0.0,1.01,0.000000,13.267327,1.000000
25,2023,6,03C907660S,12.84,1000-1,232.562724,0.0,1.01,0.322581,11.712871,0.680613
49,2023,7,03C907660S,11.62,1000-1,223.942581,0.0,1.01,0.592655,10.504950,0.413213
73,2023,8,03C907660S,10.57,1000-1,216.161316,0.0,1.01,0.385150,9.465347,0.618663


In [672]:
test = feature_processing(filled_df,'2023-10-01')
output = demods_groby_month(test).sort_values(['零件号','仓库','year','month'])
output[(output['零件号']=='KITPAB100')
                      &(output['仓库']=='8012')]

,year,month,零件号,pred_values,仓库
20,2022,6,KITPAB100,48,8012
44,2022,7,KITPAB100,8,8012
68,2022,8,KITPAB100,20,8012
92,2022,9,KITPAB100,4,8012
116,2022,10,KITPAB100,21,8012
140,2022,11,KITPAB100,14,8012
164,2022,12,KITPAB100,6,8012
188,2023,1,KITPAB100,3,8012
212,2023,2,KITPAB100,6,8012
236,2023,3,KITPAB100,8,8012


In [671]:
full_compart_info.sort_values(['零件号','month']).to_excel('预测效果_20231108.xlsx')

In [667]:
full_compart_info.chumi_mape.mean()

0.8294235510261331

In [668]:
full_compart_info.svg_mape.mean()
# 1.提高模型响应速度 最近月份的比重
# 2.对话机器人的流程 chatgpt 

1.0703751347341575

In [669]:
full_compart_info.groupby(['month'])['chumi_mape','svg_mape'].mean().reset_index()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_59614/433864545.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_compart_info.groupby(['month'])['chumi_mape','svg_mape'].mean().reset_index()


,month,chumi_mape,svg_mape
0,5,1.145840,0.890008
1,6,1.039884,0.887022
2,7,0.783346,0.839089
3,8,0.778333,0.964095
4,9,0.716954,1.516228
5,10,0.512185,1.325809


In [5]:
import random  
  
def auto_dialog():  
    sentences_a = ['你真棒', '好开心', 'ojrk', '嗯嗯'] 
    sentences_b = ['操你妈', '干你娘', '傻逼', '干']  

    round_num = 0  
    while round_num < 10:  
        if round_num % 2 == 0:  
            if random.choice([True, False]):  
                print(f"小明说：{random.choice(sentences_a)}")  
            else:  
                print(f"小红说：{random.choice(sentences_b)}")  
        else:  
            if random.choice([True, False]):  
                print(f"小红说：{random.choice(sentences_b)}")  
            else:  
                print(f"小明说：{random.choice(sentences_a)}")  
        round_num += 1  
if __name__ == "__main__":  
    auto_dialog()

小明说：好开心
小明说：嗯嗯
小明说：你真棒
小明说：好开心
小明说：ojrk
小红说：操你妈
小明说：你真棒
小明说：嗯嗯
小明说：ojrk
小红说：傻逼


In [27]:
import random  
  
def auto_dialog():  
    sentences_xiaoming = ['你是不是觉得自己极度的清醒？',
'自己总是可以在第一时间拨开迷雾看到本质。觉得洞悉了人类思想发展历程，感觉掌握了人类社会运行规律',
'而我们其他人都是一群身陷囹圄而不自知的愚民。你不但有异于常人的清醒，并且还有着知识分子特有的温柔与善良',
'试图拼尽全力拯救愚昧的大众。你说的每一句话，都是你看到愚昧的普罗大众后',
'哀其不幸怒其不争扼腕痛惜最后捶胸顿足所说的真理',
'都是你觉醒后独立思考的精髓',
'你就是这么觉得自己的对吧？！',
'我是一个高尚的人，一个纯粹的人，一个有道德的人，一个脱离了低级趣味的人，一个有益于人民的人。',
'自以为自己是冷静客观的剖析的',
'你是自以为自己是忠言对吗',
'你已经气的大脑跟不上了吧。想到什么词汇就赶紧丢群里，做着自己胜利占据主动权的春秋大梦呢',
'自己不觉得自己可笑吗',
'已经精神分裂了',
'干嘛艾特我们发你自己呀',
'还是自以为自己说的是事实',
'一个草包还好意思嘲笑别人是掉书袋啊',
'太监嘲笑别人阳痿',
'只能复制黏贴我说过的话了',
'说你肚子里没货一肚子只有草包',
'活在自己想象中的世界里',
'精神胜利法',
'你有沒有一次對我好好說話？每一次都侮辱謾罵詆毀地域，我感覺我現在對你講話都不正常',
'我不是你發洩的工具，我也不接受你一直這樣和我講話。你願意繼續說我退群。和你聊天沒有任何意義',
'你很阿Q',
'已经没办法从我们说的话里面做反驳了',
'我们都是理性讨论的',
'是有的人破大防了',
'既然他说了说明你自己很清楚你是什么货色']  
    
    sentences_xiaohong = ['看看，又开始顾左右而言他了',
'以为想到一个很好的反击点',
'故作坚强却有心无力的驳斥',
'你想到这个反击点，一定废了不少功夫吧',
'一定觉得自己很厉害吧👍',
'是不是觉得自己不尴尬的回击了对方',
'但我还是要说',
'草包就是草包',
'试图在自己的遮羞布被扯下',
'做一些无力的抵抗来防止尴尬',
'却不知自己的一言一行更能显示出自己的自卑与浅薄',
'抠脚了',
'尴尬吗？',
'求求你 别说了',
'极力挽尊',
'又来了 又开始臆想了',
'又觉得自己能行了',
'你这么菜 又这么跳',
'一定吃了不少苦吧',
'早干嘛去了 你好委屈啊',
'哭鼻子了吗',
'太弱了,我get不到你的反抗',
'就像 小孩子 ，攻击反弹']  
    used_sentences_xiaoming = []  
    used_sentences_xiaohong = []  
    round_num = 0  
    while round_num < 100:  
        if round_num % 2 == 0:  
            if len(used_sentences_xiaoming) == 0:  
                print(f"kaka：{random.choice(sentences_xiaoming)}")  
            else:  
                new_sentence = random.choice([s for s in sentences_xiaoming if s not in used_sentences_xiaoming])  
                used_sentences_xiaoming.append(new_sentence)  
        else:  
            if len(used_sentences_xiaohong) == 0:  
                print(f"李庭漪：{random.choice(sentences_xiaohong)}")  
            else:  
                new_sentence = random.choice([s for s in sentences_xiaohong if s not in used_sentences_xiaohong])  
                used_sentences_xiaohong.append(new_sentence)  
        round_num += 1  

if __name__ == "__main__":  
    auto_dialog()
    

kaka：既然他说了说明你自己很清楚你是什么货色
李庭漪：一定吃了不少苦吧
kaka：自以为自己是冷静客观的剖析的
李庭漪：极力挽尊
kaka：精神胜利法
李庭漪：尴尬吗？
kaka：还是自以为自己说的是事实
李庭漪：就像 小孩子 ，攻击反弹
kaka：既然他说了说明你自己很清楚你是什么货色
李庭漪：早干嘛去了 你好委屈啊
kaka：哀其不幸怒其不争扼腕痛惜最后捶胸顿足所说的真理
李庭漪：试图在自己的遮羞布被扯下
kaka：自以为自己是冷静客观的剖析的
李庭漪：你这么菜 又这么跳
kaka：已经没办法从我们说的话里面做反驳了
李庭漪：你想到这个反击点，一定废了不少功夫吧
kaka：都是你觉醒后独立思考的精髓
李庭漪：就像 小孩子 ，攻击反弹
kaka：太监嘲笑别人阳痿
李庭漪：又来了 又开始臆想了
kaka：活在自己想象中的世界里
李庭漪：却不知自己的一言一行更能显示出自己的自卑与浅薄
kaka：自己不觉得自己可笑吗
李庭漪：以为想到一个很好的反击点
kaka：自以为自己是冷静客观的剖析的
李庭漪：哭鼻子了吗
kaka：说你肚子里没货一肚子只有草包
李庭漪：看看，又开始顾左右而言他了
kaka：你很阿Q
李庭漪：看看，又开始顾左右而言他了
kaka：哀其不幸怒其不争扼腕痛惜最后捶胸顿足所说的真理
李庭漪：就像 小孩子 ，攻击反弹
kaka：你是不是觉得自己极度的清醒？
李庭漪：故作坚强却有心无力的驳斥
kaka：试图拼尽全力拯救愚昧的大众。你说的每一句话，都是你看到愚昧的普罗大众后
李庭漪：抠脚了
kaka：还是自以为自己说的是事实
李庭漪：故作坚强却有心无力的驳斥
kaka：我不是你發洩的工具，我也不接受你一直這樣和我講話。你願意繼續說我退群。和你聊天沒有任何意義
李庭漪：一定吃了不少苦吧
kaka：你很阿Q
李庭漪：又来了 又开始臆想了
kaka：你有沒有一次對我好好說話？每一次都侮辱謾罵詆毀地域，我感覺我現在對你講話都不正常
李庭漪：又来了 又开始臆想了
kaka：只能复制黏贴我说过的话了
李庭漪：故作坚强却有心无力的驳斥
kaka：说你肚子里没货一肚子只有草包
李庭漪：又来了 又开始臆想了
kaka：你是不是觉得自己极度的清醒？
李庭漪：你想到这个反击点，一定废了不少功夫吧
kaka：我是一个高尚的人，一个纯粹的人，一个有道德的人，一个脱离了低级趣味的人，一

In [ ]:
['你是不是觉得自己极度的清醒？',
'自己总是可以在第一时间拨开迷雾看到本质。觉得洞悉了人类思想发展历程，感觉掌握了人类社会运行规律',
'而我们其他人都是一群身陷囹圄而不自知的愚民。你不但有异于常人的清醒，并且还有着知识分子特有的温柔与善良',
'试图拼尽全力拯救愚昧的大众。你说的每一句话，都是你看到愚昧的普罗大众后',
'哀其不幸怒其不争扼腕痛惜最后捶胸顿足所说的真理',
'都是你觉醒后独立思考的精髓',
'你就是这么觉得自己的对吧？！',
'我是一个高尚的人，一个纯粹的人，一个有道德的人，一个脱离了低级趣味的人，一个有益于人民的人。',
'自以为自己是冷静客观的剖析的',
'你是自以为自己是忠言对吗',
'你已经气的大脑跟不上了吧。想到什么词汇就赶紧丢群里，做着自己胜利占据主动权的春秋大梦呢',
'自己不觉得自己可笑吗',
'已经精神分裂了',
'干嘛艾特我们发你自己呀',
'还是自以为自己说的是事实',
'一个草包还好意思嘲笑别人是掉书袋啊',
'太监嘲笑别人阳痿',
'只能复制黏贴我说过的话了',
'说你肚子里没货一肚子只有草包',
'活在自己想象中的世界里',
'精神胜利法',
'你有沒有一次對我好好說話？每一次都侮辱謾罵詆毀地域，我感覺我現在對你講話都不正常',
'我不是你發洩的工具，我也不接受你一直這樣和我講話。你願意繼續說我退群。和你聊天沒有任何意義',
'你很阿Q',
'已经没办法从我们说的话里面做反驳了',
'我们都是理性讨论的',
'是有的人破大防了',
'既然他说了说明你自己很清楚你是什么货色','看看，又开始顾左右而言他了',
'以为想到一个很好的反击点',
'故作坚强却有心无力的驳斥',
'你想到这个反击点，一定废了不少功夫吧',
'一定觉得自己很厉害吧👍',
'是不是觉得自己不尴尬的回击了对方',
'但我还是要说',
'草包就是草包',
'试图在自己的遮羞布被扯下',
'做一些无力的抵抗来防止尴尬',
'却不知自己的一言一行更能显示出自己的自卑与浅薄',
'抠脚了',
'尴尬吗？',
'求求你 别说了',
'极力挽尊',
'又来了 又开始臆想了',
'又觉得自己能行了',
'你这么菜 又这么跳',
'一定吃了不少苦吧',
'早干嘛去了 你好委屈啊',
'哭鼻子了吗',
'太弱了,我get不到你的反抗',
'就像 小孩子 ，攻击反弹']